In [0]:
!du -lsm /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/*

1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/CloudConfigFiles
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/ConfigFiles
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Create DataSet.ipynb
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/ExportGraph_Unity.ipynb
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Export_TFLite.ipynb
88	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/FramePooling
8041	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Images
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Inference_Colab.ipynb
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Inference_FramePooling.ipynb
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/LabelMaps
13548	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Models
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/MultiFrameEvaluation.ipynb
1	/content/gdrive/My Drive/Masterarbeit/ObjectDetection/ObjectDetection_CloudGPU.ipynb
1	/content/gdrive/My Dri

# Mount Google Drive to access files

---
# Imports & Config  

## Config

In [0]:
import os

base_dir = "/content/gdrive/My Drive/Masterarbeit"
object_detection_dir = os.path.join(base_dir, "ObjectDetection")
image_classification = os.path.join(base_dir, "ImageClassification")

classes_dict_file = object_detection_dir + "/classes.pkl"
objects_dict_file = object_detection_dir + "/objects.pkl"
images_dict_file = object_detection_dir + "/images.pkl"

## Mount GDrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Save & Load for pkls

In [0]:
import collections
import pickle as pkl

pkl_vars = {"objects": objects_dict_file,
            "images": images_dict_file,
            "classes": classes_dict_file}

def get_OD(var, force=False):
  if not var in globals() or force:
    print("loading {} from pickle".format(var))
    try:
      with open(pkl_vars[var], "rb") as f:
        return pkl.load(f)
    except:
      return collections.OrderedDict()
  elif var in globals():
    print("using already defined {}".format(var))
    return globals()[var]
  print("Failed to find something existing, creating new OrderedDictioniary for {}".format(var))
  return collections.OrderedDict()


def save_OD(var, local_var = None):
  if local_var is not None:
    print("Saving {} from handover variable".format(var))
    with open(pkl_vars[var], "+wb") as f:
      pkl.dump(local_var, f)
  elif var in globals():
    print("Saving {} from globals".format(var))
    with open(pkl_vars[var], "+wb") as f:
      pkl.dump(globals()[var], f)
  else:
    print("Failed to find something, unable to save {}".format(var))
    
def create_class_id(class_dict, prefix):
  if len(class_dict) == 0:
    class_int = 1
  else:
    class_int = int(next(reversed(class_dict))[2:])+1
  class_id = "{}{}".format(prefix, class_int)
  return class_id, class_int

## Replace print by prettyprint

In [0]:
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2)
def print(*kwargs):
  pp.pprint(kwargs)
  

---
# Object Detection

## Imports

In [3]:
!git clone https://github.com/tensorflow/models.git /content/tf
!cd /content/tf/research; protoc object_detection/protos/*.proto --python_out=.
!cd /content/tf/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python /content/tf/research/object_detection/builders/model_builder_test.py

Cloning into '/content/tf'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 30457 (delta 9), reused 39 (delta 2), pack-reused 30411
Receiving objects: 100% (30457/30457), 510.46 MiB | 33.27 MiB/s, done.
Resolving deltas: 100% (19089/19089), done.
Checking out files: 100% (3061/3061), done.
W0904 15:06:53.139332 140698683299712 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0904 15:06:53.473242 140698683299712 deprecation_wrapper.py:119] From /content/tf/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.comp

In [2]:
import os
import sys

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
#import matplotlib.pyplot as plt
#import tempfile
#from six.moves.urllib.request import urlopen
#from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2

# For measuring the inference time.
from datetime import datetime

#for xml parsing
from bs4 import BeautifulSoup

import requests

#...
sys.path.append('tf/research')
sys.path.append('tf/research/slim')
from object_detection.utils import dataset_util, config_util
from object_detection import model_main

W0904 16:54:12.034948 140120267532160 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0904 16:54:12.063466 140120267532160 deprecation_wrapper.py:119] From tf/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0904 16:54:12.080816 140120267532160 deprecation_wrapper.py:119] From tf/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



## Flags

In [0]:
class FLAGS_OD:
  image_dir = os.path.join(object_detection_dir, "Images")
  
  xml_list_file = os.path.join(object_detection_dir, "Images", "xml_files.csv")
  
  #tf records directory
  tf_records_dir = os.path.join(object_detection_dir, "TFRecords")
  #label_map directory
  label_map_dir = os.path.join(object_detection_dir, "LabelMaps")
  
  #class level to be considered
  class_level = [0,1]
  #how often the class of an object needs to occure to be considered
  min_class_occurence = 100
  
  #dataset split %
  testing_percentage = 18
  validation_percentage = 0
  
  #resolution of images in dataset, used for resizing
  image_resolution= 300 
  #if false the images are only resized to scale, if true they are as well padded with zeros to be square
  resize_padded=True
  
  #pipeline config
  pipeline_config_dir = os.path.join(object_detection_dir, "ConfigFiles")
  
  #model checkpoints
  model_checkpoint_dir = os.path.join(object_detection_dir, "ModelCheckpoints")
  model_dir = os.path.join(object_detection_dir,"Models")
  
  save_checkpoints_steps = 1000
  save_checkpoints_secs = None
  throttle_secs = 600
  selected_folders=["_Automat"]
  
  num_samples_per_class = 100
  
  
class FLAGS_TB:
  # a temporary folder to run tensorboard on, doesn't seem to run on a Gdrive mounted folder
  tmp_graph_dir = "tmp_graph"
  
  # how many images fit horizontally in an image mosaic
  tb_mosaic_width = 5

## Create Dataset

In [0]:
def load_annotation(file_path, infos, use_gtin=False):
  """
  Load annotation file for a given image and parse xml to dict.
  Args:
    img_name: string of the image name.
    infos: dict with information that should be appended to image dict
  Returns:
    dict of the image with all infos
  """
  xml = ""
  with open(file_path, 'r') as f:
    xml = f.readlines()
  xml = ''.join([line.strip('\t') for line in xml])
  bs = BeautifulSoup(xml)
  image_filename = os.path.join(*(file_path.split(os.sep)[:-1]))
  filename = bs.filename.string.replace("\n","").replace("\t","").replace(" ","")
  image_filename = os.path.join(image_filename, filename)
  image = {"filename": image_filename.replace("\n",""),
           "size": {"width": int(bs.size.width.string),
                    "height": int(bs.size.height.string)},
           "objects": []}
  for key, val in infos.items():
    image[key] = val

  for ob in bs.find_all("object"):
    name = ob.find('name').string.replace("\n","").replace("\t","").replace(" ","")
    if use_gtin:
      name_parts = name.split("_")
      if len(name_parts) == 6:
        name = "gtin" + name_parts[-1]
          
    obj = {"name": name,
           "bndbox":{"xmin": int(ob.bndbox.xmin.string),
                     "xmax": int(ob.bndbox.xmax.string),
                     "ymin": int(ob.bndbox.ymin.string),
                     "ymax": int(ob.bndbox.ymax.string)},
           "pose":ob.pose.string,
           "truncated": bool(ob.truncated.string),
           "difficult": bool(ob.difficult.string)}
    image["objects"].append(obj)
  return image

In [0]:
def get_xml_files(root_dir, selected_folders,
                  testing_percentage, validation_percentage,
                  load_from_file = True):
  """Gets all xml files recursively inside the root directory
  and returns the file names with the path
  Args:
    root_dir: root directory as string
    selected_folders: list of folder names which should only be
        used for the dataset. If None will use all.
  Returns:
    list of tuples with file_name and path_name relative from
        root_dir
  """
  file_list = []
  
  if load_from_file:
    try:
      with open(FLAGS_OD.xml_list_file, "r") as f:
        for line in f:
          file_list.append(line.split(";"))
      return file_list
    except:
      tf.logging.info("Failed at loading file_list from file, creating new one")
  
  data_split = TrainValTestSplit(testing_percentage, validation_percentage)

  root_f = root_dir.split(os.sep)
  for path, _, files in tf.gfile.Walk(root_dir):
    path_f = path.split(os.sep)
    if not selected_folders is None and not \
        any(s_folder in path_f for s_folder in selected_folders):
      continue
    for file in files:
      if file.split(".")[-1] == "xml":
        rel_path = [f for f in path_f if f not in root_f]
        rel_path = os.path.join(*rel_path)
        file_list.append([file, rel_path, data_split.next_label()])
  
  return file_list
  

In [0]:
#file_list = get_xml_files(FLAGS_OD.image_dir, FLAGS_OD.selected_folders,
#                          FLAGS_OD.testing_percentage, FLAGS_OD.validation_percentage,
#                          False)
#with open(FLAGS_OD.xml_list_file, "a") as f:
#  for file in file_list:
#    f.write(";".join(file) + "\n")

In [0]:
class TrainValTestSplit:
  """This class helps in assigning a train, validation or test
  label while making sure the datasets have a similar distribution
  of classes. Helpful for small datasets
  """
  #TODO init with objects list to get actual dist
  def __init__(self, testing_percentage=10, validation_percentage=10):
    """
    Args:
      testing_percentage: int percentage of dataset that should go to test
      validation_percentage: int percentage of dataset that should go to validate
    """
    self.tr = testing_percentage
    self.v = validation_percentage
  #TODO leverage actual dist to adjust label if necessary  
  #TODO check that the distribution in all sets is the same
  #-- distribution over classes, shelf_type
  def next_label(self):
    percentage = np.random.randint(1,100)
    if percentage < self.tr:
      return "test"
    elif percentage < (self.tr + self.v):
      return "validate"
    else:
      return "train"

### array of nutri values

In [0]:
nutris = {
		"40111445":
	{
		"name":"mnms_gelb__45__40111445",
		"gtin":"40111445",
		"display_name":"M&M's Peanut 45g",
		"energy":"230.0",
		"energyletter":"B",
		"protein":"4.4",
		"proteinletter":"B",
		"sat_fat":"4.6",
		"sat_fatletter":"C",
		"sugar":"24.1",
		"sugarletter":"C",
		"nutriscore":"14",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"76401121":
	{
		"name":"ragusa___50__76401121",
		"gtin":"76401121",
		"display_name":"Ragusa 50g",
		"fiber":"3.0",
		"fiberletter":"A",
		"protein":"7.0",
		"proteinletter":"A",
		"sat_fat":"15.0",
		"sat_fatletter":"E",
		"sugar":"49.0",
		"sugarletter":"E",
		"energy":"576.0",
		"energyletter":"D",
		"nutriscore":"20",
		"nutriletter":"E",
		"fiberletter":"A",
	},
	"76415272":
	{
		"name":"torino___46__76415272",
		"gtin":"76415272",
		"display_name":"Torino Milk 46g",
		"fiber":"2.0",
		"fiberletter":"B",
		"protein":"8.0",
		"proteinletter":"A",
		"sat_fat":"20.0",
		"sat_fatletter":"E",
		"sugar":"47.0",
		"sugarletter":"E",
		"energy":"571.0",
		"energyletter":"D",
		"nutriscore":"21",
		"nutriletter":"E",
		"fiberletter":"B",
	},
	"5000159462129":
	{
		"name":"maltesers_teasers__35__5000159462129",
		"gtin":"5000159462129",
		"display_name":"Maltesers Teasers 35g",
		"sat_fat":"6.3",
		"sat_fatletter":"D",
		"sugar":"18.5",
		"sugarletter":"C",
		"protein":"2.6",
		"proteinletter":"B",
		"energy":"186.0",
		"energyletter":"C",
		"nutriscore":"2",
		"nutriletter":"B",
	},
	"7610046000259":
	{
		"name":"kagi___50__7610046000259",
		"gtin":"7610046000259",
		"display_name":"Kägi fret 25g",
		"energy":"559.0",
		"energyletter":"D",
		"sat_fat":"25.0",
		"sat_fatletter":"E",
		"sugar":"38.0",
		"sugarletter":"E",
		"fiber":"3.1",
		"fiberletter":"A",
		"protein":"6.4",
		"proteinletter":"A",
		"sodium":"0.1",
		"sodiumletter":"B",
		"nutriscore":"19",
		"nutriletter":"E",
	},
	"5000159459228":
	{
		"name":"twix___50__5000159459228",
		"gtin":"5000159459228",
		"display_name":"Twix 50g",
		"energy":"124.0",
		"energyletter":"A",
		"protein":"1.1",
		"proteinletter":"C",
		"sat_fat":"3.5",
		"sat_fatletter":"B",
		"sugar":"12.2",
		"sugarletter":"B",
		"sodium":"0.1732",
		"sodiumletter":"B",
		"nutriscore":"3",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"7610095013002":
	{
		"name":"zweifel_paprika__90__7610095013002",
		"gtin":"7610095013002",
		"display_name":"Zweifel Original Chips Paprika 90g",
		"sodium":"0.5",
		"sodiumletter":"D",
		"sat_fat":"2.0",
		"sat_fatletter":"A",
		"sugar":"5.0",
		"sugarletter":"A",
		"fiber":"5.0",
		"fiberletter":"A",
		"protein":"6.0",
		"proteinletter":"A",
		"energy":"544.0",
		"energyletter":"D",
		"nutriscore":"5",
		"nutriletter":"C",
	},
	"4014400924275":
	{
		"name":"toffifee___33__4014400924275",
		"gtin":"4014400924275",
		"display_name":"Storck Toffifee 33g",
		"energy":"522.0",
		"energyletter":"D",
		"protein":"6.0",
		"proteinletter":"A",
		"sodium":"0.102362205",
		"sodiumletter":"B",
		"sat_fat":"12.7",
		"sat_fatletter":"E",
		"sugar":"48.9",
		"sugarletter":"E",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"5000159418546":
	{
		"name":"balisto_muesli__37g__5000159418546",
		"gtin":"5000159418546",
		"display_name":"Balisto Muesli 37g",
		"energy":"93.0",
		"energyletter":"B",
		"protein":"1.1",
		"proteinletter":"C",
		"sat_fat":"2.0",
		"sat_fatletter":"B",
		"sugar":"8.1",
		"sugarletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"4035800488808":
	{
		"name":"knoppers_riegel____4035800488808",
		"gtin":"4035800488808",
		"display_name":"None",
		"energy":"535.0",
		"energyletter":"D",
		"sat_fat":"14.4",
		"sat_fatletter":"E",
		"sugar":"38.8",
		"sugarletter":"E",
		"protein":"8.4",
		"proteinletter":"A",
		"sodium":"0.153543309392603",
		"sodiumletter":"B",
		"nutriscore":"19",
		"nutriletter":"E",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"4008400320328":
	{
		"name":"bueno___43__4008400320328",
		"gtin":"4008400320328",
		"display_name":"Kinder Bueno",
		"protein":"8.6",
		"proteinletter":"A",
		"sugar":"41.2",
		"sugarletter":"E",
		"energy":"122.0",
		"energyletter":"A",
		"sat_fat":"17.3",
		"sat_fatletter":"E",
		"nutriscore":"12",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"7610032065170":
	{
		"name":"darwida_sandwich____7610032065170",
		"gtin":"7610032065170",
		"display_name":"DAR-VIDA Sandwich cream cheese & herbs 1 Pocket-Pack",
		"sat_fat":"6.0",
		"sat_fatletter":"D",
		"sugar":"3.0",
		"sugarletter":"B",
		"fiber":"5.0",
		"fiberletter":"A",
		"protein":"10.0",
		"proteinletter":"A",
		"sodium":"0.67",
		"sodiumletter":"E",
		"energy":"483.0",
		"energyletter":"D",
		"nutriscore":"6",
		"nutriletter":"C",
	},
	"5000159461122":
	{
		"name":"snickers___50__5000159461122",
		"gtin":"5000159461122",
		"display_name":"Snickers 50g",
		"energy":"241.0",
		"energyletter":"B",
		"protein":"4.3",
		"proteinletter":"B",
		"sat_fat":"4.0",
		"sat_fatletter":"B",
		"sugar":"26.0",
		"sugarletter":"C",
		"nutriscore":"6",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"76404160":
	{
		"name":"valser_classic__50__76404160",
		"gtin":"76404160",
		"display_name":"Valser Sparkling 0.5L PET",
		"sodium":"0.00105",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"nutriscore":"-15",
		"nutriletter":"A",
		"energyletter":"B",
		"energy":"0.0",
	},
	"3068320353500":
	{
		"name":"evian___50__3068320353500",
		"gtin":"3068320353500",
		"display_name":"evian Natural Mineral Water non-sparkling, 0.5L",
		"energy":"0.0",
		"energyletter":"A",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"sodium":"0.0065",
		"sodiumletter":"B",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"-15",
		"nutriletter":"A",
		"fiberletter":"C",
	},
	"305764335648":
	{
		"name":"volvic_pinapple__50__305764335648",
		"gtin":"305764335648",
		"energyletter":"B",
		"nutriscore":"8",
		"nutriletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter": "C",
    "sugar":"6.4",
    "sugarletter": "C",
		"fiber":"0.0",
    "fiberletter": "C",
		"protein":"0.0",
    "proteinletter": "C",
		"energy":"26.0",
      
	},
	"5449000236623":
	{
		"name":"fuse_peach__50__5449000236623",
		"gtin":"5449000236623",
		"display_name":"Fusetea Peach Hibiscus 0.5L PET",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"4.5",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"nutriscore":"6",
		"nutriletter":"D",
		"energyletter":"B",
		"fiber":"0.0",
    "fiberletter": "C",
		"energy":"19.0",
	},
	"5449000235947":
	{
		"name":"fuse_lemon__50__5449000235947",
		"gtin":"5449000235947",
		"display_name":"Fusetea Lemon Lemongrass 0.5L PET",
		"energy":"19.0",
		"energyletter":"A",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"4.4",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.012",
		"sodiumletter":"B",
		"nutriscore":"4",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"7610097111072":
	{
		"name":"rivella_rot__50__7610097111072",
		"gtin":"7610097111072",
		"display_name":"None",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"sugar":"9.0",
		"sugarletter":"D",
		"energy":"37.0",
		"energyletter":"B",
		"nutriscore":"8",
		"nutriletter":"D",
	},
	"90162909":
	{
		"name":"redbull___33__90162909",
		"gtin":"90162909",
		"display_name":"Red Bull : energy drink",
		"energy":"46.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"11.0",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"10",
		"nutriletter":"E",
	},
	"7610335001530":
	{
		"name":"valser_vivabirne__50__7610335001530",
		"gtin":"7610335001530",
		"display_name":"Valser Viva Pear & Melissa 0.5L PET",
		"energy":"18.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.0",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"4.2",
		"sugarletter":"C",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"7610057001078":
	{
		"name":"ramseier_jusdepomme__50__7610057001078",
		"gtin":"7610057001078",
		"display_name":"None",
		"energy":"113.0",
		"energyletter":"C",
		"protein":"0.5",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"11.0",
		"sugarletter":"E",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"40822938":
	{
		"name":"fanta___50__40822938",
		"gtin":"40822938",
		"display_name":"Fanta Orange 0.33L Glas",
		"energy":"51.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.0146",
		"sodiumletter":"B",
		"fiber":"0.0",
		"fiberletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"12.2",
		"sugarletter":"E",
		"nutriscore":"11",
		"nutriletter":"E",
	},
	"54490840":
	{
		"name":"mezzomix___50__54490840",
		"gtin":"54490840",
		"display_name":"Mezzomix 0.5L PET",
		"energy":"43.0",
		"energyletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"10.5",
		"sugarletter":"E",
		"sodium":"0.0",
		"sodiumletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"5449000131836":
	{
		"name":"coke_zero_flasche_50__5449000131836",
		"gtin":"5449000131836",
		"display_name":"Coca-Cola zero sugar 0.5L PET",
		"sodium":"0.0",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"energy":"0.2",
		"energyletter":"A",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"54491472":
	{
		"name":"coke__flasche_50__54491472",
		"gtin":"54491472",
		"display_name":"Coca-Cola 0.5L PET",
		"energy":"42.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"10.6",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"C",
		"sodium":"0.045",
		"sodiumletter":"B",
		"nutriscore":"10",
		"nutriletter":"E",
	},
	"7622300336738":
	{
		"name":"oreo___154__7622300336738",
		"gtin":"7622300336738",
		"display_name":"None",
		"energy":"480.0",
		"energyletter":"D",
		"sat_fat":"9.8",
		"sat_fatletter":"D",
		"sugar":"38.0",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"B",
		"protein":"5.0",
		"proteinletter":"A",
		"sodium":"0.354330713982931",
		"sodiumletter":"C",
		"nutriscore":"20",
		"nutriletter":"E",
	},
	"4251097402635":
	{
		"name":"bifi_roll__50__4251097402635",
		"gtin":"4251097402635",
		"display_name":"None",
		"sugar":"3.5",
		"sugarletter":"A",
		"sat_fat":"14.0",
		"sat_fatletter":"E",
		"protein":"15.0",
		"proteinletter":"A",
		"fiber":"1.5",
		"fiberletter":"B",
		"energy":"450.0",
		"energyletter":"C",
		"nutriscore":"10",
		"nutriletter":"C",
	},
	"9120025930135":
	{
		"name":"c+swiss_dosenabisicetea__33__9120025930135",
		"gtin":"9120025930135",
		"display_name":"None",
		"energy":"31.0",
		"energyletter":"B",
		"sat_fat":"0",
		"sat_fatletter":"B",
		"sugar":"7.4",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.00787401586628735",
		"sodiumletter":"B",
		"nutriscore":"9",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"4000177605004":
	{
		"name":"caprisun_multivitamin__20__4000177605004",
		"gtin":"4000177605004",
		"display_name":"Capri-Sun - Multi Juice Drink Multivitamin 200ml",
		"sodium":"0.015",
		"sodiumletter":"B",
		"sat_fat":"0.1",
		"sat_fatletter":"A",
		"sugar":"9.0",
		"sugarletter":"D",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.5",
		"proteinletter":"C",
		"energy":"39.0",
		"energyletter":"B",
		"nutriscore":"8",
		"nutriletter":"D",
	},
	"7613100037253":
	{
		"name":"comella_schokodrink__33__7613100037253",
		"gtin":"7613100037253",
		"display_name":"COMELLA Choco Drink",
		"sat_fat":"0.6",
		"sat_fatletter":"B",
		"protein":"4.5",
		"proteinletter":"B",
		"sugar":"10.0",
		"sugarletter":"C",
		"energy":"66.0",
		"energyletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"7610335002575":
	{
		"name":"valser_still__50__7610335002575",
		"gtin":"7610335002575",
		"display_name":"Valser Still 0.5L PET",
		"energy":"0.0",
		"energyletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"0.0",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"fiber":"0.0",
		"fiberletter":"C",
		"sodium":"3e-05",
		"sodiumletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"90162800":
	{
		"name":"redbull_light__33__90162800",
		"gtin":"90162800",
		"display_name":"Red Bull Sugarfree 25CL",
		"energy":"3.0",
		"energyletter":"A",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"1",
		"nutriletter":"B",
	},
	"7610095184009":
	{
		"name":"zweifel_graneochilli__100__7610095184009",
		"gtin":"7610095184009",
		"display_name":"Zweifel Graneo Mild Chilli 100g",
		"fiber":"5.0",
		"fiberletter":"A",
		"energy":"471.0",
		"energyletter":"C",
		"sugar":"7.0",
		"sugarletter":"A",
		"protein":"8.0",
		"proteinletter":"A",
		"sat_fat":"1.5",
		"sat_fatletter":"A",
		"sodium":"0.95",
		"sodiumletter":"E",
		"nutriscore":"8",
		"nutriletter":"C",
	},
	"4047751730219":
	{
		"name":"jacklinks_beefjerkyorginal__25__4047751730219",
		"gtin":"4047751730219",
		"display_name":"None",
		"energy":"262.0",
		"energyletter":"C",
		"sat_fat":"1.7",
		"sat_fatletter":"B",
		"sugar":"12.0",
		"sugarletter":"C",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"42.0",
		"proteinletter":"A",
		"sodium":"1.9",
		"sodiumletter":"E",
		"nutriscore":"4",
		"nutriletter":"C",
	},
	"5000159023061":
	{
		"name":"maltesers___100__5000159023061",
		"gtin":"5000159023061",
		"display_name":"Maltesers Classic 100g",
		"energy":"167.0",
		"energyletter":"B",
		"protein":"2.7",
		"proteinletter":"B",
		"sat_fat":"5.0",
		"sat_fatletter":"C",
		"sugar":"17.2",
		"sugarletter":"B",
		"nutriscore":"13",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"4018077798818":
	{
		"name":"lorenz_nicnacs__40__4018077798818",
		"gtin":"4018077798818",
		"display_name":"None",
		"fiber":"4.4",
		"fiberletter":"A",
		"sat_fat":"11.0",
		"sat_fatletter":"E",
		"sugar":"7.3",
		"sugarletter":"A",
		"energy":"540.0",
		"energyletter":"D",
		"protein":"15.0",
		"proteinletter":"A",
		"nutriscore":"9",
		"nutriletter":"C",
	},
	"7610200279682":
	{
		"name":"malburner_partysticks__40__7610200279682",
		"gtin":"7610200279682",
		"display_name":"None",
		"nutriscore":"12",
		"nutriletter":"D",
		"energyletter":"D",
		"sat_fat":"14.0",
    "sat_fatletter":"E",
		"sugar":"1",
    "sugarletter":"B",
		"fiber":"0.0",
    "fiberletter":"C",
		"protein":"36.0",
    "proteinletter":"A",
		"energy":"490.0",
	},
	"4029679520028":
	{
		"name":"powerbar_proteinplusschoko__55__4029679520028",
		"gtin":"4029679520028",
		"display_name":"PowerBar ProteinPlus Bar 30% 55g",
		"energy":"187.0",
		"energyletter":"C",
		"protein":"17.0",
		"proteinletter":"A",
		"sat_fat":"2.2",
		"sat_fatletter":"C",
		"sugar":"16.0",
		"sugarletter":"C",
		"fiber":"7.4",
		"fiberletter":"A",
		"sodium":"0.1",
		"sodiumletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"50173167":
	{
		"name":"airwaves_menthoneucalyptus_riegel_14_1_50173167",
		"gtin":"50173167",
		"display_name":"Airwaves M&E 14g 10P",
		"energy":"142.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"0.0",
		"sugarletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"57060330":
	{
		"name":"stimorol_wildcherry_riegel_14_1_57060330",
		"gtin":"57060330",
		"display_name":"None",
		"energy":"182.0",
		"energyletter":"C",
		"protein":"0.4",
		"proteinletter":"C",
		"sat_fat":"0.4",
		"sat_fatletter":"B",
		"sugar":"0.1",
		"sugarletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"8410525150364":
	{
		"name":"fini_galaxymix_packung_100_1_8410525150364",
		"gtin":"8410525150364",
		"display_name":"None",
		"energy":"320.0",
		"energyletter":"C",
		"protein":"5.0",
		"proteinletter":"A",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"54.0",
		"sugarletter":"E",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"8410525116704":
	{
		"name":"fini_jellykisses_packung_80_1_8410525116704",
		"gtin":"8410525116704",
		"display_name":"None",
		"sugar":"58.0",
		"sugarletter":"E",
		"energy":"326.4",
		"energyletter":"C",
		"protein":"4.0",
		"proteinletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"5449000235947":
	{
		"name":"fuse_lemon_dose_33__5449000235947",
		"gtin":"5449000235947",
		"display_name":"Fusetea Lemon Lemongrass 0.5L PET",
		"energy":"19.0",
		"energyletter":"A",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"4.4",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.012",
		"sodiumletter":"B",
		"nutriscore":"4",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"54491472":
	{
		"name":"coke__dose_33__54491472",
		"gtin":"54491472",
		"display_name":"Coca-Cola 0.5L PET",
		"energy":"42.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"10.6",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"C",
		"sodium":"0.045",
		"sodiumletter":"B",
		"nutriscore":"10",
		"nutriletter":"E",
	},
	"3057640335648":
	{
		"name":"volvic_teeminze__50__3057640335648",
		"gtin":"3057640335648",
		"display_name":"Volvic Thé - Grüntee Minze 0.5L",
		"sodium":"0.004",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"4.9",
		"sugarletter":"B",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.5",
		"proteinletter":"C",
		"energy":"21.0",
		"energyletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"7610238230617":
	{
		"name":"schweppes_citrus__50__7610238230617",
		"gtin":"7610238230617",
		"display_name":"Schweppes Citrus Fusion PET 0,5l",
		"energy":"35",
		"sugar":"8.5",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"B",
		"sat_fatletter":"B",
		"proteinletter":"C",
	},
	"0000000000001":
	{
		"name":"tiki_himbeerbrause_dose_33_1_",
		"gtin":"0000000000001",
		"display_name":"Tiki Himbeerbrause 0,25l",
		"energy":"47",
		"sugar":"11",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"C",
		"sat_fatletter":"B",
		"proteinletter":"C",
	},
	"0000000000002":
	{
		"name":"icedcoffee_???____",
		"gtin":"0000000000002",
		"display_name":"Icedcoffee ???",
		"energy":"20",
		"sugar":"5",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"C",
		"sat_fatletter":"B",
		"proteinletter":"C",
	},
	"0000000000003":
	{
		"name":"7days_croissantschoko_packung_80_1_",
		"gtin":"0000000000003",
		"display_name":"7days Croissant Schokolade 80g",
		"energy":"440",
		"sugar":"19",
		"sat_fat":"18",
		"protein":"7",
		"fiber":"3",
    "fiberletter": "B",
		"energyletter":"D",
		"sugarletter":"C",
		"sat_fatletter":"E",
		"proteinletter":"A",
	},
	"0000000000004":
	{
		"name":"skittles_sour_riegel_51_1_",
		"gtin":"0000000000004",
		"display_name":"Skittles Sour 51g",
		"energy":"398",
		"sugar":"87.4",
		"sat_fat":"2.4",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"C",
		"sugarletter":"E",
		"sat_fatletter":"C",
		"proteinletter":"C",
	},
	"0000000000005":
	{
		"name":"stimorol_spearmint_riegel_14_1_",
		"gtin":"0000000000005",
		"display_name":"Stimorol Spearmint Chewingum 2Pack",
		"energy":"160",
		"sugar":"0.1",
		"sat_fat":"0.3",
		"protein":"0.4",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"B",
		"sat_fatletter":"B",
		"proteinletter":"C",
	},
	"5410495002079":
	{
		"name":"waffel_schoko____5410495002079",
		"gtin":"5410495002079",
		"display_name":"JL Bernard Gauffre Maxi Liegeoise Cacao",
		"energy":"450",
		"sugar":"28",
		"sat_fat":"15",
		"protein":"6",
		"fiber":"2",
    "fiberletter": "B",
		"energyletter":"D",
		"sugarletter":"D",
		"sat_fatletter":"E",
		"proteinletter":"A",
	},
	"4002160092334":
	{
		"name":"granini_orange__50__4002160092334",
		"gtin":"4002160092334",
		"display_name":"Granini Orangejuice PET 0.2l",
		"energy":"43",
		"sugar":"9",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "B",
		"veggieratio":"0.5",
		"energyletter":"C",
		"sugarletter":"D",
		"sat_fatletter":"B",
		"proteinletter":"C",
	},
	"0000000000008":
	{
		"name":"milka_tender____",
		"gtin":"0000000000008",
		"display_name":"Milka Tender Milchriegel",
		"energy":"425",
		"sugar":"37.5",
		"sat_fat":"12.5",
		"protein":"6",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"D",
		"sugarletter":"E",
		"sat_fatletter":"E",
		"proteinletter":"A",
	},
}

### ...

In [0]:
def update_classes(objects, classes):
  """Checks if for a given class naming level all objects are
  assigned the correct class and stored in classes
  Args:
    objects: the objects dict
    classes: the classes dict
  Returns:
    updated objects and classes
  """
  for obj_id, obj in objects.items():
    if not "classes" in obj:
      obj["classes"] = []
    cls_nm = obj["name"]
    cls_id = None
    for cl_id, cl in classes.items():
      if cl["name"] == cls_nm:
        cls_id = cl_id
    if cls_id is None:
      cls_id, it = create_class_id(classes, "c_")
      if cls_nm[0:4] == "gtin":
        gtin = cls_nm[4:]
      else:
        gtin = cls_nm.split("_")[-1]
      if gtin in nutris:
        display_name = nutris[gtin]["name"]
      else:
        response = requests.get("https://eatfit-service.foodcoa.ch/products/{}/".format(gtin),
                             auth=('eatfit_student_tobias', '6swafchC2nvpBwvy'))
        if response.status_code == 200 and \
            'products' in response.json() and \
            isinstance(response.json()['products'], list) and \
            len(response.json()['products']) > 0:
          products = response.json()['products']
          if not products[0]["product_name_en"] == "None":
            display_name = products[0]["product_name_en"]
          else:
            display_name = products[0]["product_name_de"]
      if not display_name or display_name is None:
        display_name = cls_nm
        print("missing class: ",cls_nm)
      display_name = display_name.replace("ä","ae").replace("ü","ue").replace("ö","oe")
      display_name = display_name.replace(" ","")
      classes[cls_id] = {"name": cls_nm,
                         "display_name": cls_nm,
                         "cnt": 0,
                         "objects": [],
                         "id": it}
    if not obj_id in classes[cls_id]["objects"]:
      classes[cls_id]["cnt"] += 1
      classes[cls_id]["objects"].append(obj_id)
    if not cls_id in obj["classes"]:
      obj["classes"].append(cls_id)
  
  return objects, classes

In [0]:
def create_image_dict(file_list, image_dir):
  """Update the list of images and object annotations from the file system.
  Split them into stable training, testing, and validation sets.
  Args:
    image_dir: String path to a folder containing subfolders of images.
    testing_percentage: Integer percentage of the images to reserve for tests.
    validation_percentage: Integer percentage of images reserved for validation.
    images: a dict containing all the information per image
    objects: a dict containing all the information per object
    classes: a dict containing all the information per class
  Returns:
    an updated or new images, objects & classes dict
  """

  images = dict()
  num_files = 0
  for file_name, rel_path, set_type in file_list:
    infos = {"type": set_type,
             "shelf_type": rel_path.split(os.sep)[0]}
    images[file_name] = load_annotation(os.path.join(image_dir,rel_path,file_name), infos)
    #img_id, _ = create_class_id(images, "i_")
    #objects_list = []
    #for obj in image["objects"]:
    #  obj_id, _ = create_class_id(objects, "o_")
    #  objects[obj_id] = obj
    #  objects[obj_id]["image"] = img_id
    #  objects_list.append(obj_id)
    #image["objects"] = objects_list
    #images[img_id] = image
    num_files += 1
    
    if (num_files) % 20 == 0:
      print("images found sofar: {}".format(num_files))
  print("images found total: {}".format(num_files))    
  #objects, classes = update_classes(objects, classes)
  
  return images #, objects, classes

In [10]:
file_list = get_xml_files("/content/gdrive/My Drive/Masterarbeit/Data/Images",
                          ["_Automat"],
                          0,
                          0,
                          False)

images = create_image_dict(file_list,
                           "/content/gdrive/My Drive/Masterarbeit/Data/Images")

images found sofar: 20
images found sofar: 40
images found sofar: 60
images found sofar: 80
images found sofar: 100
images found sofar: 120
images found sofar: 140
images found sofar: 160
images found sofar: 180
images found sofar: 200
images found sofar: 220
images found sofar: 240
images found sofar: 260
images found sofar: 280
images found total: 295


In [11]:
import shutil

for key, val in images.items():
  print(key)
  img_key = val["filename"].split("/")[-1]
  src = "/content/gdrive/My Drive/Masterarbeit/Data/Images/_Automat/"
  dst = "/content/gdrive/My Drive/Masterarbeit/Data/Images/FinalDataset/"
  shutil.copyfile(src + key, dst + key)
  shutil.copyfile(src + img_key, dst + img_key)

IMG_20181218_171802.xml
IMG_20190125_113823.xml
IMG_20190306_172008.xml
IMG_20190306_173403.xml
DSC01671.xml
DSC01669.xml
DSC01672.xml
DSC01633.xml
DSC01675.xml
DSC01655.xml
DSC01667.xml
DSC01678.xml
DSC01679.xml
DSC01635.xml
DSC01663.xml
IMG_20181218_130011.xml
IMG_20181218_130005.xml
IMG_20181218_130002.xml
DSC01632.xml
DSC01634.xml
DSC01656.xml
IMG_20190206_170714.xml
DSC01657.xml
IMG_20190206_170716.xml
DSC01658.xml
IMG_20190206_174446.xml
DSC01659.xml
DSC01660.xml
DSC01661.xml
DSC01662.xml
DSC01664.xml
DSC01665.xml
DSC01666.xml
DSC01668.xml
DSC01670.xml
DSC01673.xml
DSC01674.xml
DSC01676.xml
DSC01677.xml
DSC01680.xml
DSC01681.xml
IMG_20181218_164606.xml
IMG_20181218_164608.xml
IMG_20190430_111611.xml
IMG_20190502_093212.xml
IMG_20190430_111949.xml
IMG_20190430_111818.xml
IMG_20190430_111942.xml
IMG_20190502_093643.xml
IMG_20190502_093933.xml
IMG_20190502_094330.xml
IMG_20190502_093957.xml
IMG_20190502_093929.xml
IMG_20190502_094333.xml
20190111_111616_HoloLens.xml
20190111_111631_

In [15]:
import pickle as pkl
with open("/content/gdrive/My Drive/Masterarbeit/Data/Images/FinalDataset/images.pkl", "wb+") as file:
  pkl.dump(images, file)

RecursionError: ignored

In [0]:
def create_tf_example(img_width, img_height, img_filename, img_encoded_data,
                      img_format, objects):
  """Creates a tf example for the data
  Args:
    img_height: image height in pxl
    img_width: image widht in pxl
    img_filename: filename of the image
    img_encoded_data: encoded image bytes
    objects: either list of dicts or dict of lists of objects in the image
        dict must contain:
        "xmin": x position of box corner top left
        "xmax": x position of box corner bottom right
        "ymin": y position of box corner top left
        "ymax": y position of box corner bottom right
        "class_name": name of class as string
        "class": name of class as continuous int id
  Returns:
    tf_example: a tf example for the Record Writer
  """
  #for convience accept both
  #TODO use labelmap instead to assign class
  if isinstance(objects, list):
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    for obj in objects:
      xmins.append(obj["xmin"])
      xmaxs.append(obj["xmax"])
      ymins.append(obj["ymin"])
      ymaxs.append(obj["ymax"])
      classes_text.append(obj["class_name"].encode('utf-8'))
      classes.append(obj["class"])
  elif isinstance(objects, dict):
    xmins = objects["xmin"]
    xmaxs = objects["xmax"]
    ymins = objects["ymin"]
    ymaxs = objects["ymax"]
    classes_text = objects["class_name"].encode('utf-8')
    classes = objects["class"]
  else:
    return -1
  
  feature_dict = {
      'image/height': dataset_util.int64_feature(img_height),
      'image/width': dataset_util.int64_feature(img_width),
      'image/filename': dataset_util.bytes_feature(img_filename.encode('utf-8')),
      'image/source_id': dataset_util.bytes_feature(img_filename.encode('utf-8')),
      'image/encoded': dataset_util.bytes_feature(img_encoded_data),
      'image/format': dataset_util.bytes_feature(img_format.encode('utf-8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }
  tf_example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
  return tf_example

In [0]:
def create_label_map(images, label_map_path, use_gtin_only=False):
  """uses the classes dict to create a label_map as required by the object detection API
  Args:
    classes
  Returns:
    label_map
  """
  classes = {}
  it = 1
  for key, val in images.items():
    for obj in val["objects"]:
      if not obj["name"] in classes:
        classes[obj["name"]] = {"name":obj["name"],
                                "id": it,
                                "cnt": 0}
        it += 1
      classes[obj["name"]]["cnt"] += 1
        
  
  label_map = {}
  with open(label_map_path, "w+") as f:
    for key, val in classes.items():
      add = True
      if val["cnt"] <= FLAGS_OD.min_class_occurence:
        add = False
      if use_gtin_only and val["name"][0:4] != "gtin":
        add = False
      
      if add:
        f.write('item {\n')
        f.write('  name: "{}"\n'.format(val["name"]))
        #f.write('  display_name: "{}"\n'.format(val["display_name"]))
        f.write('  id: {}\n'.format(val["id"]))
        f.write('}\n')

        label_map[key] = {"class": val["id"],
                          "class_name": val["name"]}
  
  print("classes:")
  for cl_id, cl in classes.items():
    if cl_id in label_map:
      print("\t", cl["name"], " - ", cl["cnt"])
  return label_map

In [0]:
def resize_padded_with_bndboxes(width_old, height_old, filepath_old, bndboxs_old, length):
  """resizes the image ot a squared padded image. Where the longest
  side will match their target length. The other dimension is padded
  equally on both sides with zeros until it as well matches length.
  Args:
    width_old: the old width
    height_old: the old height
    filepath_old: the old filepath
    bndboxs: a list of bounding boxes
    length: target length
  Returns:
    width: the new width
    length: the new length
    filepath: the new filepath
    bndboxs: the new boundingboxes
  """
  #First resize the image
  width_old, height_old, filepath, bndboxs_old = resize_scaled_with_bndboxes(width_old,
                                                                 height_old,
                                                                 filepath_old,
                                                                 bndboxs_old,
                                                                 length)
  
  #necessary padding, handle uneven numbers
  pad_x = 0
  pad_y = 0
  if height_old < length:
    pad_size = length - height_old
    pad_x = int(pad_size/2)
  if width_old < length:
    pad_size = length - width_old
    pad_y = int(pad_size/2)
  
  #create empty image array and embedd the image in it
  image = np.zeros([length, length, 3])
  image_old = cv2.imread(filepath)
  image[pad_x:(pad_x + height_old), pad_y:(pad_y + width_old), :] = image_old
  cv2.imwrite(filepath, image)
  
  #redo boundingboxes
  bndboxs = []
  for bndbox in bndboxs_old:
    bndboxs.append({})
    bndboxs[-1]["xmin"] = (bndbox["xmin"] * width_old + pad_y) / float(length)
    bndboxs[-1]["xmax"] = (bndbox["xmax"] * width_old + pad_y) / float(length)
    bndboxs[-1]["ymin"] = (bndbox["ymin"] * height_old  + pad_x) / float(length)
    bndboxs[-1]["ymax"] = (bndbox["ymax"] * height_old  + pad_x) / float(length)
    bndboxs[-1]["class_name"] = bndbox["class_name"]
    bndboxs[-1]["class"] = bndbox["class"]
    
    if not all([(float(val) >= 0.0 and float(val) <= 1.0) for key, val in bndboxs[-1].items() if key in ["xmin", "xmax", "ymin", "ymax"]]):
      tf.logging.info("bndboxes out of scope!!")
      tf.logging.info(bndboxs[-1])
      for key, val in bndboxs[-1].items():
        if key in ["xmin", "xmax", "ymin", "ymax"]:
          if val > 1.0:
            val = 1.0
          elif val < 0.0:
            val = 0.0
  
  return length, length, filepath, bndboxs

In [0]:
def resize_scaled_with_bndboxes(width_old, height_old, filepath_old, bndboxs_old, max_length):
  """resize the image and the bounding boxes accordingly
  It checks if a resized image already exists and otherwise
  creates one. the boundingbox coordinates are normalized
  for calculated since they need to be normalized later on
  anyway.
  Args:
    width_old: the old width
    height_old: the old height
    filepath_old: the old filepath
    bndboxs: a list of bounding boxes
    max_length = the size in pixel of the longest side
  Returns:
    width: the new width
    length: the new length
    filepath: the new filepath
    bndboxs: the new boundingboxes
    
  """
  #find a scaler from the longest side and resize the image
  if width_old > height_old:
    scaler = width_old / max_length
  else:
    scaler = height_old / max_length
  
  height = int(height_old / scaler) 
  width = int(width_old / scaler)
  cv_img = cv2.imread(filepath_old, cv2.IMREAD_COLOR)
  cv_img_resized = cv2.resize(cv_img, (width,height))
  filename = filepath_old.split(os.sep)[-1]
  filename = ".".join(filename.split(".")[:-1]) + "_resized.{}".format(filename.split(".")[-1])
  if not os.path.exists("tmp"):
    os.mkdir("tmp")
  filepath = os.path.join("tmp", filename)
  cv2.imwrite(filepath, cv_img_resized)
  
  bndboxs = []
  for bndbox in bndboxs_old:
    bndboxs.append({})
    bndboxs[-1]["xmin"] = bndbox["xmin"] / float(width_old)
    bndboxs[-1]["xmax"] = bndbox["xmax"] / float(width_old)
    bndboxs[-1]["ymin"] = bndbox["ymin"] / float(height_old)
    bndboxs[-1]["ymax"] = bndbox["ymax"] / float(height_old)
    bndboxs[-1]["class_name"] = bndbox["class_name"]
    bndboxs[-1]["class"] = bndbox["class"]
    
    if not all([(float(val) >= 0.0 and float(val) <= 1.0) for key, val in bndboxs[-1].items() if key in ["xmin", "xmax", "ymin", "ymax"]]):
      tf.logging.info("bndboxes out of scope!!")
      tf.logging.info(bndboxs[-1])
      for key, val in bndboxs[-1].items():
        if key in ["xmin", "xmax", "ymin", "ymax"]:
          if val > 1.0:
            val = 1.0
          elif val < 0.0:
            val = 0.0
  
  return width, height, filepath, bndboxs

In [0]:
def clear_according_to_labelmap(images, label_map):
  num_cleaned = 0
  tf.logging.info(label_map)
  for key, image in images.items():
    objects_new = []
    for it, obj in enumerate(image["objects"]):
      if obj["name"] in label_map:
        objects_new.append(obj)
    image["objects"] = objects_new
    num_cleaned += 1
    if num_cleaned % 20 == 0:
      tf.logging.info("cleaned {} images".format(num_cleaned))
  return images

In [0]:
from random import random

def evenout_sample(images, n):
  """even outs the sample to ensure per class are only n patches
  
  """
  if n <= 0:
    n = 99999999999
  
  max_cnts = {}
  for filename, img in images.items():
    if not img["type"] == "train":
      continue
    for obj in img["objects"]:
      if not obj["name"] in max_cnts:
        max_cnts[obj["name"]] = 0
      max_cnts[obj["name"]] += 1
  
  ps = {}
  for cls_name, cnt in max_cnts.items():
    ps[cls_name] = float(n) / cnt
    
  cnts = {obj_name:0 for obj_name in max_cnts}
  cycle_it = 0
  objects_keep = {}
  while not all([cnt == min(n, max_cnts[obj_name]) for obj_name, cnt in cnts.items()]):
    cycle_it += 1
    if cycle_it % 100 == 0:
      tf.logging.info("doing cycle: {}".format(cycle_it))
    if cycle_it % 1000 == 0:
      break
    for filename, img in images.items():
      if not img["type"] == "train":
        continue
      if cycle_it == 1:
        objects_keep[filename] = []
      for obj in img["objects"]:
        if (cnts[obj["name"]] < n and
            random() < ps[obj["name"]] and 
            not obj in objects_keep[filename]):
          objects_keep[filename].append(obj)
          cnts[obj["name"]] += 1
  
  for filename, img in images.items():
    if not img["type"] == "train":
      continue
    img["blackouts"] = [obj for obj in img["objects"] if not obj in objects_keep[filename]]
    img["objects"] = objects_keep[filename]
  
  return images

In [0]:
import cv2

def draw_black_patch(filename, blackouts):
  img = cv2.imread("/" + filename)
  for obj in blackouts:
    bb = obj["bndbox"]
    img[bb["ymin"]:bb["ymax"], bb["xmin"]:bb["xmax"],:] = 0
  
  filename = filename.split(os.sep)[-1]
  filename = ".".join(filename.split(".")[:-1]) + "_blackedout.{}".format(filename.split(".")[-1])
  if not os.path.exists("tmp"):
    os.mkdir("tmp")
  filename = os.path.join("tmp", filename)
  cv2.imwrite(filename, img)
  return filename

In [0]:
def clean_without_box(images):
  delete_keys = []
  for filename, image in images.items():
    if len(image["objects"]) == 0:
      delete_keys.append(filename)
  for key in delete_keys:
    images.pop(key, None)
      
  return images

### Main

In [0]:
file_list = get_xml_files(FLAGS_OD.image_dir,
                               FLAGS_OD.selected_folders,
                               FLAGS_OD.testing_percentage,
                               FLAGS_OD.validation_percentage,
                               False)

images = create_image_dict(file_list, FLAGS_OD.image_dir)

timestamp = "2019_6_15_15_31_42"
FLAGS_OD.min_class_occurence = 1
label_map = create_label_map(images,os.path.join(FLAGS_OD.label_map_dir, "{}.pbtxt".format(timestamp)))
tf.logging.info("Found {} classes in label map".format(len(label_map)))

print("len(images): ", len(images))

#images = clear_according_to_labelmap(images, label_map)


images found sofar: 20
images found sofar: 40
images found sofar: 60
images found sofar: 80
images found sofar: 100
images found sofar: 120
images found sofar: 140
images found sofar: 160
images found sofar: 180
images found sofar: 200
images found sofar: 220
images found sofar: 240
images found sofar: 260
images found sofar: 280
images found total: 295
classes:
	 zweifel_paprika__90__7610095013002  -  184
	 zweifel_graneochilli__100__7610095184009  -  155
	 knoppers_riegel____4035800488808  -  396
	 jacklinks_beefjerkyorginal__25__4047751730219  -  159
	 maltesers___100__5000159023061  -  164
	 kagi___50__7610046000259  -  175
	 lorenz_nicnacs__40__4018077798818  -  165
	 malburner_partysticks__40__7610200279682  -  185
	 bueno___43__4008400320328  -  158
	 oreo___66__84100733  -  171
	 snickers___50__5000159461122  -  231
	 twix___50__5000159459228  -  188
	 bifi_roll__50__4251097402635  -  188
	 caprisun_multivitamin__20__4000177605004  -  373
	 redbull___33__90162909  -  880
	 redb

In [0]:
len(label_map)

112

In [0]:
images_t = clear_according_to_labelmap(images, label_map)

classes:
	 zweifel_paprika__90__7610095013002  -  184
	 zweifel_graneochilli__100__7610095184009  -  155
	 knoppers_riegel____4035800488808  -  396
	 jacklinks_beefjerkyorginal__25__4047751730219  -  159
	 maltesers___100__5000159023061  -  164
	 kagi___50__7610046000259  -  175
	 lorenz_nicnacs__40__4018077798818  -  165
	 malburner_partysticks__40__7610200279682  -  185
	 bueno___43__4008400320328  -  158
	 oreo___66__84100733  -  171
	 snickers___50__5000159461122  -  231
	 twix___50__5000159459228  -  188
	 bifi_roll__50__4251097402635  -  188
	 caprisun_multivitamin__20__4000177605004  -  373
	 redbull___33__90162909  -  880
	 redbull_light__33__90162800  -  118
	 coke__flasche_50__54491472  -  481
	 powerbar_proteinplusschoko__55__4029679520028  -  110
	 airwaves_menthoneucalyptus_riegel_14_1_50173167  -  143
	 stimorol_wildcherry_riegel_14_1_57060330  -  143
	 fini_galaxymix_packung_100_1_8410525150364  -  148
	 fini_jellykisses_packung_80_1_8410525116704  -  162
	 fuse_lemon_do

In [0]:
prods = {
  "7days_croissantschoko_packung_80_1_0000000000003":0,
  "airwaves_menthoneucalyptus_riegel_14_1_50173167":0,
  "balisto_muesli__37g__5000159418546":0,
  "berger_zitronetoertli__50__7610404511007":0,
  "berger_spitzbueb__74__7610404591009":0,
  "bifi_roll__50__4251097402635":0,
  "bueno___43__4008400320328":0,
  "c+swiss_dosenabisicetea__33__9120025930135":1,
  "caferoyal_classicmacchiato__33__7617014164487":0,
  "caprisun_multivitamin__20__4000177605004":1,
  "coke_zero_flasche_50__5449000131836":1,
  "coke__dose_33__54491472":1,
  "coke___50__54491472":1,
  "comella_schokodrink__33__7613100037253":0,
  "darwida_sandwich____7610032065170":0,  
  "evian___50__3068320353500":2,  
  "fanta___50__40822938":1,
  "fini_galaxymix_packung_100_1_8410525150364":0,
  "fini_jellykisses_packung_80_1_8410525116704":0,
  "fuse_peach__50__5449000236623":1,
  "fuse_lemon__50__5449000235947":1,
  "fuse_lemon_dose_33__5449000235947":1,
  "granini_orange__50__4002160092334":1, 
  "henniez_rot__33__7610235000442":2,
  "henniez_gruen__33__7610235000442":2,
  "henniez_blau__33__7610235000442":2,
  "jacklinks_beefjerkyorginal__25__4047751730219":0,
  "jlbrichard_gauffrechoko__90__5410495002079":0,
  "jlbrichard_gauffre__90__5410495002000":0,
  "kagi___50__7610046000259":0,
  "kind_darkchocnutsseasalt__40__0602652176517":0,
  "knoppers_riegel____4035800488808":0,
  "lorenz_nicnacs__40__4018077798818":0,
  "malburner_partysticks__40__7610200279682":0,
  "maltesers___100__5000159023061":0,
  "maltesers_teasers__35__5000159462129":0,
  "milka_tender____0000000000008":0,
  "mnms_gelb__45__40111445":0,
  "mezzomix___50__54490840":1,  
  "oreo___66__84100733":0,
  "pepsi_max__50__4060800105943":1,
  "powerbar_proteinplusschoko__55__4029679520028":0,
  "ragusa___50__76401121":0,
  "ramseier_jusdepomme__50__7610057001078":1,
  "redbull___33__90162909":1,
  "redbull_light__33__90162800":1,
  "rivella_rot__50__7610097111072":1,
  "schweppes_citrus__50__7610238230617":1,
  "stimorol_wildcherry_riegel_14_1_57060330":0,
  "skittles_sour_riegel_51_1_0000000000004":0,
  "snickers___50__5000159461122":0,
  "stimorol_spearmint_riegel_14_1_0000000000005":0,
  "tiki_himbeerbrause_dose_33_1_0000000000001":1,
  "toffifee___33__4014400924275":0,
  "torino___46__76415272":0,
  "twix___50__5000159459228":0,
  "valser_vivabirne__50__7610335001530":1,
  "valser_still__50__7610335002575":2,
  "valser_classic__50__76404160":2,
  "vitaminwell_reload__50__7350042716388":1,
  "volvic_teeminze__50__3057640335648":1,
  "volvic_pinapple__50__305764335648":1,
  "zweifel_graneochilli__100__7610095184009":0,
  "zweifel_paprika__90__7610095013002":0,
}

classes = {}
it = 1
for key, val in images.items():
  for obj in val["objects"]:
    if not obj["name"] in classes:
      classes[obj["name"]] = {("Total","cnt_total"):0,
                              ("Total","cnt_test"): 0,
                              ("Total","cnt_train"): 0,
                              ("Total","cnt_total"):0,
                              ("Total","cnt_test"): 0,
                              ("Total","cnt_train"): 0,
                              ("Total","cnt_total"):0,
                              ("Total","cnt_test"): 0,
                              ("Total","cnt_train"): 0}
      it += 1
    classes[obj["name"]]["cnt_total"] += 1
    if prods[obj["name"]] > 0:
      

In [0]:



for key, val in classes.items():
  

{'zweifel_paprika__90__7610095013002': {'name': 'zweifel_paprika__90__7610095013002', 'id': 1, 'cnt': 184}, 'zweifel_graneochilli__100__7610095184009': {'name': 'zweifel_graneochilli__100__7610095184009', 'id': 2, 'cnt': 155}, 'knoppers_riegel____4035800488808': {'name': 'knoppers_riegel____4035800488808', 'id': 3, 'cnt': 396}, 'jacklinks_beefjerkyorginal__25__4047751730219': {'name': 'jacklinks_beefjerkyorginal__25__4047751730219', 'id': 4, 'cnt': 159}, 'maltesers___100__5000159023061': {'name': 'maltesers___100__5000159023061', 'id': 5, 'cnt': 164}, 'kagi___50__7610046000259': {'name': 'kagi___50__7610046000259', 'id': 6, 'cnt': 175}, 'lorenz_nicnacs__40__4018077798818': {'name': 'lorenz_nicnacs__40__4018077798818', 'id': 7, 'cnt': 165}, 'malburner_partysticks__40__7610200279682': {'name': 'malburner_partysticks__40__7610200279682', 'id': 8, 'cnt': 185}, 'bueno___43__4008400320328': {'name': 'bueno___43__4008400320328', 'id': 9, 'cnt': 158}, 'oreo___66__84100733': {'name': 'oreo___66

In [0]:
def main_create_tfrecords_dataset(_):  
  
  now = datetime.now()
  #timestamp = "_".join([str(now.year), str(now.month), str(now.day), str(now.hour), str(now.minute), str(now.second)])
  timestamp = "2019_6_15_15_31_42"
  tf.logging.info(timestamp)
  
  for num_samples in [1,3,6]:
  
    file_list = get_xml_files(FLAGS_OD.image_dir,
                               FLAGS_OD.selected_folders,
                               FLAGS_OD.testing_percentage,
                               FLAGS_OD.validation_percentage,
                               False)

    images = create_image_dict(file_list, FLAGS_OD.image_dir)
    
    label_map = create_label_map(images,os.path.join(FLAGS_OD.label_map_dir, "{}.pbtxt".format(timestamp)))
    tf.logging.info("Found {} classes in label map".format(len(label_map)))

    print("len(images): ", len(images))

    images = clear_according_to_labelmap(images, label_map)
  
    FLAGS_OD.num_samples_per_class = num_samples
    images = evenout_sample(images, FLAGS_OD.num_samples_per_class)
    
    images = clean_without_box(images) 
    tf.logging.info("after evenout {} images are left in training".format(len([1 for _,image in images.items() if image["type"] == "train"])))
    
    for resolutions in [320]:
      
      train_cnt = 0
      val_cnt = 0
      test_cnt = 0
      
      FLAGS_OD.image_resolution = resolutions
  
      test_writer = tf.python_io.TFRecordWriter(os.path.join(FLAGS_OD.tf_records_dir, "{}_test_{}_{}.record".format(timestamp, FLAGS_OD.image_resolution, FLAGS_OD.num_samples_per_class)))
      validate_writer = tf.python_io.TFRecordWriter(os.path.join(FLAGS_OD.tf_records_dir, "{}_validate_{}_{}.record".format(timestamp, FLAGS_OD.image_resolution, FLAGS_OD.num_samples_per_class)))
      train_writer = tf.python_io.TFRecordWriter(os.path.join(FLAGS_OD.tf_records_dir, "{}_train_{}_{}.record".format(timestamp, FLAGS_OD.image_resolution, FLAGS_OD.num_samples_per_class)))

      for key, image in images.items():
        print("working on image: ", key)

        bndboxs = []
        for obj in image["objects"]:
          bndboxs.append(obj["bndbox"])
          bndboxs[-1]["class"] = label_map[obj["name"]]["class"]
          bndboxs[-1]["class_name"] = label_map[obj["name"]]["class_name"]

        if image["type"] == "train":
          filename = draw_black_patch(image["filename"], image["blackouts"])
        else:
          filename = "/" + image["filename"]

        if FLAGS_OD.resize_padded:
          width, height, filename, bndboxs = resize_padded_with_bndboxes(image["size"]["width"],
                                                                         image["size"]["height"],
                                                                         filename,
                                                                         bndboxs,
                                                                         FLAGS_OD.image_resolution)
        else:
          width, height, filename, bndboxs = resize_scaled_with_bndboxes(image["size"]["width"],
                                                                         image["size"]["height"],
                                                                         filename,
                                                                         bndboxs,
                                                                         FLAGS_OD.image_resolution)

        with tf.gfile.Open(filename, "rb") as file:
          encoded_jpg = file.read()

        tf_example = create_tf_example(width,
                                       height,
                                       image["filename"],
                                       encoded_jpg,
                                       'png',
                                       bndboxs)
        print("writing to: {}".format(image["type"]))
        if image["type"] == "test":
          test_writer.write(tf_example.SerializeToString())
          test_cnt += 1
        elif image["type"] == "validate":
          validate_writer.write(tf_example.SerializeToString())
          val_cnt += 1
        elif image["type"] == "train":
          train_writer.write(tf_example.SerializeToString())
          train_cnt += 1
        else:
          tf.logging.info("unknown type skipping {}".format(image["filename"]))

      test_writer.close()
      validate_writer.close()
      train_writer.close()

      print("num elements in train: ", train_cnt)
      print("num elements in val: ", val_cnt)
      print("num elements in test: ", test_cnt)

In [0]:
## in case the files need to be sharded / split
#import contextlib2
#from google3.third_party.tensorflow_models.object_detection.dataset_tools import tf_record_creation_util
#
#num_shards=10
#output_filebase='/path/to/train_dataset.record'
#
#with contextlib2.ExitStack() as tf_record_close_stack:
#  output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
#      tf_record_close_stack, output_filebase, num_shards)
#  for index, example in examples:
#    tf_example = create_tf_example(example)
#    output_shard_index = index % num_shards
#    output_tfrecords[output_shard_index].write(tf_example.SerializeToString())

### App Run

In [0]:
#content/gdrive/My Drive/Masterarbeit/ObjectDetection/Images/images.pkl

#resolution of images in dataset, used for resizing
FLAGS_OD.image_resolution= 320 
#if false the images are only resized to scale, if true they are as well padded with zeros to be square
FLAGS_OD.resize_padded=True

FLAGS_OD.num_samples_per_class = 1000
tf.app.run(main_create_tfrecords_dataset)

I0616 17:47:29.722753 139796394403712 <ipython-input-21-eae388f72821>:6] 2019_6_15_15_31_42


images found sofar: 20
images found sofar: 40
images found sofar: 60
images found sofar: 80
images found sofar: 100
images found sofar: 120
images found sofar: 140
images found sofar: 160
images found sofar: 180
images found sofar: 200
images found sofar: 220
images found sofar: 240
images found sofar: 260
images found sofar: 280
images found total: 295


I0616 17:49:50.795266 139796394403712 <ipython-input-21-eae388f72821>:19] Found 37 classes in label map
I0616 17:49:50.796852 139796394403712 <ipython-input-17-5ac99fda931d>:3] {'zweifel_paprika__90__7610095013002': {'class': 1, 'class_name': 'zweifel_paprika__90__7610095013002'}, 'zweifel_graneochilli__100__7610095184009': {'class': 2, 'class_name': 'zweifel_graneochilli__100__7610095184009'}, 'knoppers_riegel____4035800488808': {'class': 3, 'class_name': 'knoppers_riegel____4035800488808'}, 'jacklinks_beefjerkyorginal__25__4047751730219': {'class': 4, 'class_name': 'jacklinks_beefjerkyorginal__25__4047751730219'}, 'maltesers___100__5000159023061': {'class': 5, 'class_name': 'maltesers___100__5000159023061'}, 'kagi___50__7610046000259': {'class': 6, 'class_name': 'kagi___50__7610046000259'}, 'lorenz_nicnacs__40__4018077798818': {'class': 7, 'class_name': 'lorenz_nicnacs__40__4018077798818'}, 'malburner_partysticks__40__7610200279682': {'class': 8, 'class_name': 'malburner_partysticks_

classes:
	 zweifel_paprika__90__7610095013002  -  184
	 zweifel_graneochilli__100__7610095184009  -  155
	 knoppers_riegel____4035800488808  -  396
	 jacklinks_beefjerkyorginal__25__4047751730219  -  159
	 maltesers___100__5000159023061  -  164
	 kagi___50__7610046000259  -  175
	 lorenz_nicnacs__40__4018077798818  -  165
	 malburner_partysticks__40__7610200279682  -  185
	 bueno___43__4008400320328  -  158
	 oreo___66__84100733  -  171
	 snickers___50__5000159461122  -  231
	 twix___50__5000159459228  -  188
	 bifi_roll__50__4251097402635  -  188
	 caprisun_multivitamin__20__4000177605004  -  373
	 redbull___33__90162909  -  880
	 redbull_light__33__90162800  -  118
	 coke__flasche_50__54491472  -  481
	 powerbar_proteinplusschoko__55__4029679520028  -  110
	 airwaves_menthoneucalyptus_riegel_14_1_50173167  -  143
	 stimorol_wildcherry_riegel_14_1_57060330  -  143
	 fini_galaxymix_packung_100_1_8410525150364  -  148
	 fini_jellykisses_packung_80_1_8410525116704  -  162
	 fuse_lemon_do

I0616 17:52:23.398307 139796394403712 <ipython-input-21-eae388f72821>:19] Found 37 classes in label map
I0616 17:52:23.399807 139796394403712 <ipython-input-17-5ac99fda931d>:3] {'zweifel_paprika__90__7610095013002': {'class': 1, 'class_name': 'zweifel_paprika__90__7610095013002'}, 'zweifel_graneochilli__100__7610095184009': {'class': 2, 'class_name': 'zweifel_graneochilli__100__7610095184009'}, 'knoppers_riegel____4035800488808': {'class': 3, 'class_name': 'knoppers_riegel____4035800488808'}, 'jacklinks_beefjerkyorginal__25__4047751730219': {'class': 4, 'class_name': 'jacklinks_beefjerkyorginal__25__4047751730219'}, 'maltesers___100__5000159023061': {'class': 5, 'class_name': 'maltesers___100__5000159023061'}, 'kagi___50__7610046000259': {'class': 6, 'class_name': 'kagi___50__7610046000259'}, 'lorenz_nicnacs__40__4018077798818': {'class': 7, 'class_name': 'lorenz_nicnacs__40__4018077798818'}, 'malburner_partysticks__40__7610200279682': {'class': 8, 'class_name': 'malburner_partysticks_

images found total: 295
classes:
	 zweifel_paprika__90__7610095013002  -  184
	 zweifel_graneochilli__100__7610095184009  -  155
	 knoppers_riegel____4035800488808  -  396
	 jacklinks_beefjerkyorginal__25__4047751730219  -  159
	 maltesers___100__5000159023061  -  164
	 kagi___50__7610046000259  -  175
	 lorenz_nicnacs__40__4018077798818  -  165
	 malburner_partysticks__40__7610200279682  -  185
	 bueno___43__4008400320328  -  158
	 oreo___66__84100733  -  171
	 snickers___50__5000159461122  -  231
	 twix___50__5000159459228  -  188
	 bifi_roll__50__4251097402635  -  188
	 caprisun_multivitamin__20__4000177605004  -  373
	 redbull___33__90162909  -  880
	 redbull_light__33__90162800  -  118
	 coke__flasche_50__54491472  -  481
	 powerbar_proteinplusschoko__55__4029679520028  -  110
	 airwaves_menthoneucalyptus_riegel_14_1_50173167  -  143
	 stimorol_wildcherry_riegel_14_1_57060330  -  143
	 fini_galaxymix_packung_100_1_8410525150364  -  148
	 fini_jellykisses_packung_80_1_8410525116704

I0616 17:55:37.668182 139796394403712 <ipython-input-21-eae388f72821>:19] Found 37 classes in label map
I0616 17:55:37.670132 139796394403712 <ipython-input-17-5ac99fda931d>:3] {'zweifel_paprika__90__7610095013002': {'class': 1, 'class_name': 'zweifel_paprika__90__7610095013002'}, 'zweifel_graneochilli__100__7610095184009': {'class': 2, 'class_name': 'zweifel_graneochilli__100__7610095184009'}, 'knoppers_riegel____4035800488808': {'class': 3, 'class_name': 'knoppers_riegel____4035800488808'}, 'jacklinks_beefjerkyorginal__25__4047751730219': {'class': 4, 'class_name': 'jacklinks_beefjerkyorginal__25__4047751730219'}, 'maltesers___100__5000159023061': {'class': 5, 'class_name': 'maltesers___100__5000159023061'}, 'kagi___50__7610046000259': {'class': 6, 'class_name': 'kagi___50__7610046000259'}, 'lorenz_nicnacs__40__4018077798818': {'class': 7, 'class_name': 'lorenz_nicnacs__40__4018077798818'}, 'malburner_partysticks__40__7610200279682': {'class': 8, 'class_name': 'malburner_partysticks_

images found total: 295
classes:
	 zweifel_paprika__90__7610095013002  -  184
	 zweifel_graneochilli__100__7610095184009  -  155
	 knoppers_riegel____4035800488808  -  396
	 jacklinks_beefjerkyorginal__25__4047751730219  -  159
	 maltesers___100__5000159023061  -  164
	 kagi___50__7610046000259  -  175
	 lorenz_nicnacs__40__4018077798818  -  165
	 malburner_partysticks__40__7610200279682  -  185
	 bueno___43__4008400320328  -  158
	 oreo___66__84100733  -  171
	 snickers___50__5000159461122  -  231
	 twix___50__5000159459228  -  188
	 bifi_roll__50__4251097402635  -  188
	 caprisun_multivitamin__20__4000177605004  -  373
	 redbull___33__90162909  -  880
	 redbull_light__33__90162800  -  118
	 coke__flasche_50__54491472  -  481
	 powerbar_proteinplusschoko__55__4029679520028  -  110
	 airwaves_menthoneucalyptus_riegel_14_1_50173167  -  143
	 stimorol_wildcherry_riegel_14_1_57060330  -  143
	 fini_galaxymix_packung_100_1_8410525150364  -  148
	 fini_jellykisses_packung_80_1_8410525116704

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
import shutil
import os

timestamp = "2019_5_20_22_4_17"

with open(os.path.join(FLAGS_OD.tf_records_dir,"{}.md".format(timestamp)), "+w") as f:
  f.write("# DataSet Description  \n")
  f.write("selected_folders: {}  \n".format(FLAGS_OD.selected_folders))
  f.write("Image Resolution: 1000 x 1000  \n")
  f.write("Image Scaled to Size: True  \n")
  f.write("Image Padded: True  \n")
  f.write("Classes: 47  \n")
  f.write("Num Training Images: 246  \n")
  f.write("Num Val Images: 0  \n")
  f.write("Num Test Images: 50  \n")

print("created ", timestamp, "MD")

created  2019_5_20_22_4_17 MD


## Check TFRecord

In [0]:
from object_detection.utils import visualization_utils

def draw_per_frame(image, boxes, classes):
  """for each image draw the boxes and the labels, with the color according
  to the nutriscore
  
  """
  boxes = np.reshape(boxes,[int(len(classes)),4])
  classes = np.squeeze(classes).astype(np.int32)
  use_normalized_coordinates=True
  line_thickness=3
  nutri_colors = ["Green", "LimeGreen", "Yellow", "Orange", "Red"]
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  for i in range(0, boxes.shape[0]):
    box = tuple(boxes[i].tolist())
    display_str = '{}'.format(classes[i])
    box_to_color_map[box] = nutri_colors[np.random.randint(0,5)]
    box_to_display_str_map[box].append(display_str)
        
  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    visualization_utils.draw_bounding_box_on_image_array(
        image,
        ymin,
        xmin,
        ymax,
        xmax,
        color=color,
        thickness=line_thickness,
        display_str_list=box_to_display_str_map[box],
        use_normalized_coordinates=use_normalized_coordinates)

  return image

ModuleNotFoundError: ignored

In [0]:
%matplotlib inline
#import IPython.display as display
import io
import tensorflow as tf
import PIL

from matplotlib.pyplot import figure, imshow, show
import numpy as np
import collections


record_iterator = tf.python_io.tf_record_iterator(path="/content/gdrive/My Drive/Masterarbeit/ObjectDetection/TFRecords/2019_5_28_9_24_26_test_320_1000.record")

print_boxes = True

it = 0
for string_record in record_iterator:
  print(it)
  it += 1
  
  example = tf.train.Example()
  example.ParseFromString(string_record)
  filename = example.features.feature["image/filename"].bytes_list.value[0].decode('utf-8')
  
  if print_boxes:
    #print(dict(example.features.feature).keys())
    print(example.features.feature["image/filename"].bytes_list.value[0].decode('utf-8'))
    #print(example.features.feature["image/height"].int64_list.value)
    #print(example.features.feature["image/width"].int64_list.value)
    #print(type(example.features.feature["image/encoded"].bytes_list.value))
    encoded_jpg_io = io.BytesIO(example.features.feature["image/encoded"].bytes_list.value[0])
    pil_image = PIL.Image.open(encoded_jpg_io)
    print(pil_image.size)

    num_boxes = len(example.features.feature['image/object/bbox/xmin'].float_list.value)
    print(num_boxes)
    boxes = np.zeros((num_boxes,4))
    classes = np.zeros((num_boxes,1))
    for it in range(0, num_boxes):
      boxes[it, 1] = example.features.feature['image/object/bbox/xmin'].float_list.value[it]
      boxes[it, 3] = example.features.feature['image/object/bbox/xmax'].float_list.value[it]
      boxes[it, 0] = example.features.feature['image/object/bbox/ymin'].float_list.value[it]
      boxes[it, 2] = example.features.feature['image/object/bbox/ymax'].float_list.value[it]
      classes[it] = example.features.feature['image/object/class/label'].int64_list.value[it]

    image_tmp = np.asarray(pil_image)
    image = np.zeros(image_tmp.shape)
    image[:,:,:] = image_tmp[:,:,:]
    image.setflags(write=True)
    image_with_boxes = draw_per_frame(image, boxes, classes)
    fig = figure()
    imshow(image_with_boxes / 255.0)
    show()